In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages databricks:tensorframes:0.2.7-s_2.11 pyspark-shell'

In [4]:
import findspark
findspark.init("/home/canwill/spark2/")
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("TensorOnSpark") \
    .config("spark.executor.memory", "5g") \
    .getOrCreate()
    
spark.conf.set("spark.sql.caseSensitive", "false");
spark.sql("set spark.sql.caseSensitive=false");

In [5]:
import tensorflow as tf
import tensorframes as tfs

In [6]:
from pyspark.sql import Row
from pyspark.sql.functions import *

In [7]:
data = spark.read.csv("titanic3.csv", header=True)

In [8]:
data.columns

['pclass',
 'survived',
 'name',
 'sex',
 'age',
 'sibsp',
 'parch',
 'ticket',
 'fare',
 'cabin',
 'embarked',
 'boat',
 'body',
 'home.dest']

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline
import random
import numpy as np
import pandas as pd
from sklearn import datasets, svm, cross_validation, tree, preprocessing, metrics
import sklearn.ensemble as ske
import tensorflow as tf
from tensorflow.contrib import learn

/home/canwill/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
classGrps = data.groupBy(['pclass','sex']).mean().show()

+------+------+
|pclass|   sex|
+------+------+
|     2|  male|
|     2|female|
|     3|  male|
|     1|  male|
|     1|female|
|     3|female|
+------+------+



In [19]:
titanic_df = data.drop('body','cabin','boat', 'home.dest')

In [20]:
titanic_df.show()

+------+--------+--------------------+------+------+-----+-----+--------+--------+--------+
|pclass|survived|                name|   sex|   age|sibsp|parch|  ticket|    fare|embarked|
+------+--------+--------------------+------+------+-----+-----+--------+--------+--------+
|     1|       1|Allen, Miss. Elis...|female|    29|    0|    0|   24160|211.3375|       S|
|     1|       1|Allison, Master. ...|  male|0.9167|    1|    2|  113781|151.5500|       S|
|     1|       0|Allison, Miss. He...|female|     2|    1|    2|  113781|151.5500|       S|
|     1|       0|Allison, Mr. Huds...|  male|    30|    1|    2|  113781|151.5500|       S|
|     1|       0|Allison, Mrs. Hud...|female|    25|    1|    2|  113781|151.5500|       S|
|     1|       1| Anderson, Mr. Harry|  male|    48|    0|    0|   19952| 26.5500|       S|
|     1|       1|Andrews, Miss. Ko...|female|    63|    1|    0|   13502| 77.9583|       S|
|     1|       0|Andrews, Mr. Thom...|  male|    39|    0|    0|  112050|  0.000

In [21]:
 titanic_df = titanic_df.na.fill("NA")

In [22]:
 titanic_df = titanic_df.na.drop()

In [24]:
titanic_df.count()

1309

In [ ]:
data = [Row(x=[float(x), float(2 * x)], key=str(x % 2), z = float(x+1)) for x in range(1, 6)]
df = spark.createDataFrame(data)

In [ ]:
tfs.print_schema(df)

In [ ]:
df2 = tfs.analyze(df)
tfs.print_schema(df2)

In [ ]:
df.show()

In [ ]:
data = [Row(x=float(x)) for x in range(5)]
df = spark.createDataFrame(data)
with tf.Graph().as_default() as g:
    # The placeholder that corresponds to column 'x'
    x = tf.placeholder(tf.double, shape=[None], name="x")
    # The output that adds 3 to x
    z = tf.add(x, 3, name='z')
    # The resulting dataframe
    df2 = tfs.map_blocks(z, df)

df2.show()

In [ ]:
with tf.Graph().as_default() as g:
    x = tfs.block(df, "x")
    z = tf.add(x, 3, name='z')
    df2 = tfs.map_blocks(z, df)

df2.show()

In [ ]:
with tf.Graph().as_default() as g:
    # The placeholder that corresponds to rows in column 'x'
    x = tf.placeholder(tf.double, shape=[], name="x")
    z = tf.add(x, 3, name='z')
    df2 = tfs.map_rows(z, df)

In [ ]:
with tf.Graph().as_default() as g:
    # The placeholder that corresponds to rows in column 'x'
    x = tfs.row(df, "x")
    z = tf.add(x, 3, name='z')
    df2 = tfs.map_rows(z, df)

In [ ]:
data = [Row(x=float(x)) for x in range(5)]
df = spark.createDataFrame(data)

with tf.Graph().as_default() as g:
    # The placeholders that correspond to column 'x'.
    # Note the convention of calling them with '_1' and '_2'
    x_1 = tf.placeholder(tf.double, shape=[], name="x_1")
    x_2 = tf.placeholder(tf.double, shape=[], name="x_2")
    # We sum the two inputs.
    x = tf.add(x_1, x_2, name='x')
    # The resulting number
    res = tfs.reduce_rows(x, df)

In [ ]:
from __future__ import print_function

import numpy as np
import tflearn

In [ ]:
# Download the Titanic dataset
from tflearn.datasets import titanic
titanic.download_dataset('titanic_dataset.csv')

In [ ]:
# Load CSV file, indicate that the first column represents labels
from tflearn.data_utils import load_csv
data, labels = load_csv('titanic_dataset.csv', target_column=0,
                        categorical_labels=True, n_classes=2)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("titanic_dataset.csv")
df

In [ ]:
data

In [ ]:
# Preprocessing function
def preprocess(passengers, columns_to_delete):
    # Sort by descending id and delete columns
    for column_to_delete in sorted(columns_to_delete, reverse=True):
        [passenger.pop(column_to_delete) for passenger in passengers]
    for i in range(len(passengers)):
        # Converting 'sex' field to float (id is 1 after removing labels column)
        passengers[i][1] = 1. if data[i][1] == 'female' else 0.
    return np.array(passengers, dtype=np.float32)


In [ ]:
# Ignore 'name' and 'ticket' columns (id 1 & 6 of data array)
to_ignore=[1, 6]

data = preprocess(data, to_ignore)

In [ ]:
data

In [ ]:
# Build neural network
net = tflearn.input_data(shape=[None, 6])
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

In [ ]:
# Define model
model = tflearn.DNN(net)

In [ ]:
# Start training (apply gradient descent algorithm)
model.fit(data, labels, n_epoch=10, batch_size=16, show_metric=True)

In [ ]:
# Let's create some data for DiCaprio and Winslet
dicaprio = [3, 'Jack Dawson', 'male', 109, 0, 0, 'N/A', 5.0000]
winslet = [1, 'Rose DeWitt Bukater', 'female', 17, 1, 2, 'N/A', 100.0000]

In [ ]:
# Preprocess data
dicaprio, winslet = preprocess([dicaprio, winslet], to_ignore)

In [ ]:
# Predict surviving chances (class 1 results)
pred = model.predict([dicaprio, winslet])

In [ ]:
print("DiCaprio Surviving Rate:", pred[0][1])
print("Winslet Surviving Rate:", pred[1][1])

In [ ]:
Training Step: 820  | total loss: 0.48264 | time: 0.192s
| Adam | epoch: 010 | loss: 0.48264 - acc: 0.8062 -- iter: 1309/1309

In [ ]:
DiCaprio Surviving Rate: 0.167573690414
Winslet Surviving Rate: 0.554718375206